In [2]:
#!pip install surprise
import surprise 
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k') 
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD()
algo.fit(trainset)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [3]:
predictions = algo.test(testset) # test : 사용자-아이템 평점 데이터 세트 전체에 대해서 추천 예측 
print('prediction type :', type(predictions), ' size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.73808468370287, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.859612974136778, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.283121018295666, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.2367309726566194, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.45941132803054, details={'was_impossible': False})]

In [4]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid) # predict : 사용자, 아이템 정보를 입력하여 추천 예측 평점 
print(pred)

user: 196        item: 302        r_ui = None   est = 4.14   {'was_impossible': False}


In [5]:
accuracy.rmse(predictions)

RMSE: 0.9488


0.9487981919403441

In [8]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors = 50, random_state = 0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8928


0.8927792463064537

In [9]:
from surprise.model_selection import cross_validate 

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8940  0.8894  0.8988  0.8987  0.8875  0.8937  0.0046  
MAE (testset)     0.6866  0.6868  0.6880  0.6914  0.6808  0.6867  0.0034  
Fit time          2.35    2.25    2.24    2.21    2.26    2.26    0.05    
Test time         0.07    0.06    0.07    0.07    0.07    0.07    0.00    


{'fit_time': (2.345184564590454,
  2.2496633529663086,
  2.244626760482788,
  2.2094640731811523,
  2.259730577468872),
 'test_mae': array([0.68657926, 0.68676905, 0.68796574, 0.69143691, 0.6808191 ]),
 'test_rmse': array([0.89400218, 0.88943858, 0.89884753, 0.89866428, 0.88752417]),
 'test_time': (0.06537175178527832,
  0.06223607063293457,
  0.07145094871520996,
  0.07245182991027832,
  0.06939125061035156)}

In [10]:
from surprise.model_selection import GridSearchCV
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8923446740040832
{'n_epochs': 20, 'n_factors': 50}


In [11]:
from surprise.dataset import DatasetAutoFolds

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file='./ml-latest-small/ratings_noh.csv', reader=reader)
trainset = data_folds.build_full_trainset()

algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

movies = pd.read_csv('movies.csv')
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId'] == 42])

uid = str(9)
iid = str(42)
pred = algo.predict(uid, iid, verbose = True)

def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    total_movies = movies['movieId'].tolist()
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), '전체 영화수:',len(total_movies))
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)



AttributeError: ignored